In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
import time

/home/ubuntu/proj/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_model = "./model/new_model/"
base_model = "./model/base_model/"

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=compute_dtype,
    bnb_8bit_use_double_quant=False,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [01:42<00:00, 51.39s/it]


In [9]:
#bg_model = PeftModel.from_pretrained(model, new_model)
#bg_model = bg_model.merge_and_unload()

ValueError: Cannot merge LORA layers when the model is loaded in 8-bit mode

In [10]:
tokenizer = AutoTokenizer.from_pretrained(new_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [11]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
def generate(text, pipeline=pipe):
  # Ignore warnings

  start_time = time.time()
  logging.set_verbosity(logging.CRITICAL)

  # Run text generation pipeline with our next model
  prompt = text
  result = pipeline(f"[INST] {prompt} [/INST]")
  end_time = time.time()
  execution_time = end_time - start_time
  print("Execution time:",execution_time)
  print(result[0]['generated_text'])

In [17]:
generate("Какво да си сготвя днес? Имам картофи, месо. подправки.")

Execution time: 33.94137620925903
[INST] Какво да си сготвя днес? Имам картофи, месо. подправки. [/INST] Начин на приготвяне на Картофи с месо: Картофите се сваряват и се нарязват на филийки. Месото се сварява и се нарязва на малки парчета. В тава се нареждат картофите, отгоре се слага месото и се поръсва с подправките. Залива се с вода и се пече до готовност. 10 минути преди да се извади от фурната се поръсва с магданоз. 10 минути след като се извади от фурната се поръсва с чесън. 10 минути


In [18]:
generate("Знаеш ли как мога да тренирам невронна мрежа?")

Execution time: 34.296059370040894
[INST] Знаеш ли как мога да тренирам невронна мрежа? [/INST]  Който е в състояние да се простира в тази течност, може да се тренират невронни мрежи с помощта на следните подходи: 1.1. Мозък: 1.1.1. Мозъкът е най-добрият подход за трениране на невронни мрежи. 1.1.2. Мозъкът е най-добрият подход за трениране на невронни мрежи. 1.2. Компютер: 1.2.1. Компютерът е най-добрият подход за трениране на невронни мрежи. 1.
